In [1]:
import cv2
import torch
import time

from utils.inference.image_processing import crop_face
from utils.inference.video_processing import read_video, get_final_video_frame, add_audio_from_another_video
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_single import Face_detect_crop
from arcface_model.iresnet import iresnet100

/home/jovyan/conda/envs/faceshift_env/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


### Load Models

In [ ]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# основная модель для генерации
G = AEI_Net(c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_0_035000_init_arch_arcface2.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# модель arcface для того, чтобы достать эмбеддинг лица
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# модель, которая позволяет находить точки лица
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

### Set here path to source image and video for faceswap

In [ ]:
"""
choose not really long videos, coz it can take a lot of time othervise 
choose source image as a photo -- preferable a selfie of a person
"""

path_to_video = 'examples/videos/dora_short.mp4'
source_full = cv2.imread('examples/images/elon_musk.jpg')
OUT_VIDEO_NAME = "examples/results/elon2dora.mp4"
crop_size = 224 # don't change this

In [ ]:
# check, if we can detect face on the source image

try:
    source = crop_face(source_full, app, crop_size)[0]
    source = source[:, :, ::-1]
    print("Everything is ok!")
except TypeError:
    print("Bad source image. Choose another one.")

In [ ]:
# read video

full_frames, fps = read_video(path_to_video)

### Model Inference

In [ ]:
START_TIME = time.time()

In [ ]:
final_frames, crop_frames, full_frames, tfm_array = model_inference(full_frames,
                                                                    source,
                                                                    [netArc],
                                                                    G,
                                                                    app, 
                                                                    crop_size=crop_size)

In [ ]:
get_final_video_frame(final_frames,
                      crop_frames,
                      full_frames,
                      tfm_array,
                      OUT_VIDEO_NAME,
                      fps, 
                      handler)

In [ ]:
add_audio_from_another_video(path_to_video, OUT_VIDEO_NAME, "audio")

In [ ]:
print(f'Full pipeline took {time.time() - START_TIME}')
print(f"Video saved with path {OUT_VIDEO_NAME}")